In [1]:
import boto3
import numpy as np
from constants import BUCKET, DATA_PROCESSED_PREFIX

# Create s3 client
s3 = boto3.client("s3",
    region_name="us-west-1",
)

In [2]:
train_X = np.genfromtxt("data/train_X.csv", delimiter=",")
train_Y = np.genfromtxt("data/train_Y.csv", delimiter=",")
test_X = np.genfromtxt("data/test_X.csv", delimiter=",")
test_Y = np.genfromtxt("data/test_Y.csv", delimiter=",")

## Baseline Model

## FaNcY Model

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
# from train.model import LSTMClassifier
# from train.train import train

In [4]:
train_data = torch.from_numpy(train_X).long()
train_labels = torch.from_numpy(train_Y[:,0]).float()  # Take first label

In [27]:
# Turn the input pandas dataframe into tensors
train_sample_y = train_labels[:250]
train_sample_X = train_data[:250,:]

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=10)

In [59]:
class LSTMClassifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        """Create a RNN model by setting up the layers.
        
        Parameters
        ----------
        vocab_size : int
            size of word dictionary
        embedding_dim: int
            size of embedding layer
        hidden_dim: int
            size of hidden layer
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.linear = nn.Linear(in_features=hidden_dim, out_features=1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        """Conduct forward pass."""
#         x = x.t()  # input_dim, batch_size
        x = self.embedding(x)  # batch_size, input_dim, embedding_dim
        x, _ = self.lstm(x)
        x = self.linear(x)
        return self.activation(x.squeeze())

In [60]:
# Sample
model = LSTMClassifier(10000, 100, 5000)  # 10000 is the size of word dictionary
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()
# train(model, train_sample_dl, 5, optimizer, loss_fn)

In [61]:
epochs=5
train_loader=train_sample_dl
# device = "cuda" if torch.cuda.is_available() else "cpu"
device="cpu"
print("Using {} device".format(device))

model.to(device)
for epoch in range(1, epochs + 1):
    model.train()
    total_loss = 0
    for batch in train_loader:         
        batch_X, batch_y = batch

        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        output = model(batch_X)
        loss = loss_fn(output, batch_y)

        # Backward pass and calculate gradients
        optimizer.zero_grad()
        loss.backward()
        # Clip gradient in case it explodes
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        # Update parameters
        optimizer.step()

        total_loss += loss.item()
    print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

Using cpu device


/opt/conda/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 800])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


ValueError: Target and input must have the same number of elements. target nelement (10) != input nelement (8000)

In [ ]:
batch_y.shape

In [37]:
output.shape

torch.Size([800, 10])

In [54]:
batch_X.shape

torch.Size([10, 800])